<a href="https://colab.research.google.com/github/vyperid/Qdrant-vs-ChromaDB/blob/main/ChromaDBTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Splitting into "chunks"***

In [1]:
!pip install -q langchain-text-splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 15.5 MB/s eta 0:00:00


In [2]:
!pip install -q PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 35.6 MB/s eta 0:00:00


In [3]:
import pymupdf

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
def pdf_to_txt(pdf_path, output_path):
  document = pymupdf.open(pdf_path)

  text = ""

  for page in range(len(document)):
    page = document.load_page(page)
    text += page.get_text()

  with open(output_path, "w", encoding='utf-8') as f:
    f.write(text)

pdf_path = '/content/62f12134d4d3a-dead-poets-society.pdf'
txt_path = 'deadpoets.txt'
pdf_to_txt(pdf_path, txt_path)

In [7]:
with open("/content/deadpoets.txt") as f:
  document = f.read()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=True,
    separators=["\."]
)

In [9]:
text_chunks = text_splitter.split_text(document)

In [10]:
text_chunks[:4]

['DEAD POETS SOCIETY\n“Mr. Keating?” Neil called after him. “Sir? O Captain! My\nCaptain?” Keating stopped and waited for the boys to catch up\nwith him. “What was the Dead Poets Society, sir?” Neil asked.\nFor a split second, Keating’s face reddened',
 '. “What was the Dead Poets Society, sir?” Neil asked.\nFor a split second, Keating’s face reddened. “I was just looking\nin an old annual,” Neil explained, “and...”\n“Nothing wrong with research,” Keating said, regaining his\ncomposure.\nThe boys waited for him to say more',
 '.\nThe boys waited for him to say more. “But what was it?” Neil\npressed.\nKeating looked around to make sure that no one was\nwatching. “A secret organization,” he almost whispered. “I\ndon’t know how the present administration would look upon it,\nbut I doubt the reaction would be favorable',
 '.” His eyes scanned\nthe campus as the boys held their breaths']

***Creating the "embeddings"***

In [11]:
!pip install -q chromadb langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (p

In [12]:
!pip install -q langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.9 MB/s eta 0:00:00


In [13]:
!pip install -q tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.6 MB/s eta 0:00:00


In [14]:
import os
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [15]:
os.environ["OPENAI_API_KEY"] = ""

In [16]:
embedding_function = OpenAIEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [17]:
chromaDBTest = Chroma.from_texts(
    texts=text_chunks,
    embedding=embedding_function)

In [ ]:
# Test questions
question = "What did Werra tell TechCrunch in an email interview?"
question2 = "Can you explain the training process and dataset used for the StarCoder model?"
question3 = "What did they learn from the releases?"
question4 = "Can the code from the Stack be copied?"

In [18]:
# Dead poets society test questions
questions = [
    "Who is going to be Neil Perry's roommate?",
    "What was the Dead Poets Society according to Mr. Keating?",
    "How does Headmaster Nolan emphasize the importance of the four pillars of Welton Academy?"
]

In [19]:
for question in questions:
  result = chromaDBTest.similarity_search_with_score(question, k=1)
  print(result)

[(Document(page_content='.\n“I hear we’re going to be roommates,” he said. “I’m Neil\nPerry.”\n“Todd Anderson,” he replied softly. The boys walked in\nawkward silence.\n“Why’d you leave Balincrest?” Neil asked.\n“My brother went here.”\nNeil shook his head. “Oh, so you’re THAT Anderson.”\nTodd shrugged and groaned'), 0.3023855984210968)]
[(Document(page_content='. The Dead Poets Society\nwas alive and thriving and ready to seize the day.\nThe boys left the cave reluctantly and got back to campus just\nin time for practice. “Say, look who’s the soccer instructor,”\nPitts said, as they spotted Mr. Keating approaching the field'), 0.21351230144500732)]
[(Document(page_content='.” Nolan\npaused for the applause that followed.\n“New students,” he continued, directing his attention toward\nthe newest boys to join the ranks of Welton Academy, “the key\nto your success rests on the four pillars. This applies to seventh\ngraders and transfer students alike'), 0.23881149291992188)]
